In [11]:
import pandas as pd
# from xlsxwriter.utility import xl_rowcol_to_cell
from openpyxl import load_workbook
from pathlib import Path

from sqlalchemy import create_engine, func, desc
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.exc import NoResultFound
from models import Program, Block, Workout, Workout_set, Exercise

In [12]:
engine = create_engine(f"sqlite:///data/db/gym_database.db")
Session = sessionmaker(bind=engine)
session = Session()

In [13]:
FILE = "/mnt/c/Users/gonza/OneDrive/Gym/routines_log/test2.xlsx"
# FILE = "/mnt/d/OneDrive/Gym/routines_log/test2.xlsx"
BLOCKS = ["Micro0", "Micro1", "Micro2"]

# book = load_workbook(FILE)
with pd.ExcelWriter(FILE, engine="openpyxl") as writer:
    # writer.book = book
    # print(writer.book.sheetnames)
    for block in BLOCKS:
        if block in writer.book.sheetnames:
            print(f"{block} already in excel file!")
        else:
            writer.book.create_sheet(block)
            # writer.book.create_sheet?


In [14]:
# pd.read_sql(
#     session.query(Workout_set.workout_set_id.label("ID"),
#                   Exercise.exercise_desc.label("Ejercicio"),
#                   Workout_set.set_id.label("Serie"),
#                   Workout_set.no_reps.label("Repeticiones"),
#                   Workout_set.weight.label("Peso (kg)"),
#                   Workout_set.perc_rm.label("% 1RM"),
#                   Workout_set.min_rpe.label("RPE mín."),
#                   Workout_set.max_rpe.label("RPE máx."),
#                   Workout_set.rest_min.label("Descanso (min)"))
#     .filter(Exercise.exercise_id == Workout_set.exercise_id,
#             Workout_set.workout_id == 14)
#     .order_by(Workout_set.workout_set_id)
#     .statement,
#     session.bind)

pd.read_sql(
    session.query(Workout_set.workout_set_id.label("ID"),
                  Exercise.exercise_desc.label("Ejercicio"),
                  Workout_set.set_id.label("Serie"),
                  Workout_set.no_reps.label("Repeticiones"),
                  Workout_set.weight.label("Peso (kg)"),
                  Workout_set.perc_rm.label("% 1RM"),
                  Workout_set.min_rpe.label("RPE mín."),
                  Workout_set.max_rpe.label("RPE máx."),
                  Workout_set.rest_min.label("Descanso (min)"))
    .join(Exercise.workout_sets)
    .filter(Workout_set.workout_id == 14)
    .order_by(Workout_set.workout_set_id)
    .statement,
    session.bind)

,ID,Ejercicio,Serie,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min)
0,246,peso muerto sumo,1,10,85.0,85.0,None,8,3.0
1,247,peso muerto sumo,2,10,85.0,85.0,None,8,3.0
2,248,peso muerto sumo,3,10,85.0,85.0,None,8,3.0
3,249,peso muerto sumo,4,10,85.0,85.0,None,8,3.0
4,250,hip thrust,1,10,120.0,85.0,None,8,2.0
5,251,hip thrust,2,10,120.0,85.0,None,8,2.0
6,252,hip thrust,3,10,120.0,85.0,None,8,2.0
7,253,hip thrust,1,12,100.0,75.0,None,8,2.0
8,254,hip thrust,2,12,100.0,75.0,None,8,2.0
9,255,hip thrust,3,12,100.0,75.0,None,8,2.0


In [15]:
# FILE = "/mnt/c/Users/gonza/OneDrive/Gym/routines_log/test_gym.xlsx"

with pd.ExcelWriter(FILE, engine="openpyxl") as writer:
        program_blocks = session.query(Block).filter_by(program_id=4).all()
        for program_block in program_blocks:
            block_name = program_block.block_desc
            # # Load existing excel file into current if exists...
            # if book:
            #     writer.book = book
            if block_name not in writer.book.sheetnames:
                start_row = 0
                for workout in program_block.workouts:
                    df_workout_header = pd.DataFrame([workout.date_workout, workout.workout_desc,
                                                      None, None, None],
                                                     index=["Fecha", "Descripción", "Duración (min)",
                                                            "RPE general", "Comentario general"])
                    df_workout = pd.read_sql(
                        session.query(Workout_set.workout_set_id.label("ID"),
                                      Exercise.exercise_desc.label("Ejercicio"),
                                      Workout_set.set_id.label("Serie"),
                                      Workout_set.no_reps.label("Repeticiones"),
                                      Workout_set.weight.label("Peso (kg)"),
                                      Workout_set.perc_rm.label("% 1RM"),
                                      Workout_set.min_rpe.label("RPE mín."),
                                      Workout_set.max_rpe.label("RPE máx."),
                                      Workout_set.rest_min.label("Descanso (min)"))
                        .join(Exercise.workout_sets)
                        .filter(Workout_set.workout_id == 14)
                        .order_by(Workout_set.workout_set_id)
                        .statement,
                        session.bind)

                    # Add log fields (the No. Sets, No. Reps, Weight and RPE should be replaced
                    # if needed)
                    df_workout[["¿Hecho?", "RPE", "Comentarios"]] = None

                    df_workout_header.to_excel(writer, sheet_name=block_name,
                                               startrow=start_row,
                                               index=True, header=False)
                    start_row += df_workout_header.shape[0]
                    df_workout.to_excel(writer, sheet_name=block_name,
                                        startrow=start_row,
                                        index=False)
                    start_row += (df_workout.shape[0] + 2)

In [63]:
def generate_program_excel(session, program:int, output_dir="/mnt/c/Users/gonza/OneDrive/Gym/routines_log/"):
    """
    Generates Excel file (.xlsx) with Program planning. Each program block
    is a different sheet with all the corresponding workouts.

        Parameters:
            session (SQLAlchemy.session object)
            program_id (int): Program identifier integer from database
            output_dir (str): Directory to store generated file
    """
    # If program description provided, get id
    if isinstance(program, str):
        program_id = (
            session.query(Program.program_id)
            .filter_by(program_desc=program)
            .scalar()
        )
    
    try:
        program = session.query(Program).filter_by(program_id=program_id).one()
    except NoResultFound:
        raise KeyError(f"Program_id ({program_id}) doesn't exist!")

    program_name = program.program_desc if program.program_desc else f"Program_{program.program_id}"

    with pd.ExcelWriter(output_dir + program_name + ".xlsx", engine="xlsxwriter") as writer:
        program_blocks = session.query(Block).filter_by(program_id=program_id).all()
        for program_block in program_blocks:
            block_name = program_block.block_desc
            start_row = 0
            for workout in program_block.workouts:
                df_workout_header = pd.DataFrame([workout.date_workout, workout.workout_desc], 
                                                 index=["date", "desc"])
                df_workout = pd.read_sql(
                    session.query(Exercise.exercise_desc.label("Ejercicio"),
                                func.count(Workout_set.set_id).label("Series"),
                                Workout_set.no_reps.label("Repeticiones"),
                                Workout_set.weight.label("Peso (kg)"),
                                Workout_set.perc_rm.label("% 1RM"),
                                Workout_set.min_rpe.label("RPE mín."),
                                Workout_set.max_rpe.label("RPE máx."),
                                Workout_set.rest_min.label("Descanso (min)"))
                        .group_by(Exercise.exercise_desc)
                        .filter(Exercise.exercise_id == Workout_set.exercise_id,
                                Workout_set.workout_id == workout.workout_id)
                        .order_by(Workout_set.workout_set_id)
                        .statement,
                    session.bind)
                
                # Add log fields (the No. Sets, No. Reps, Weight and RPE should be replaced
                # if needed)
                df_workout[["¿Hecho?", "Comentarios", "RPE total", 
                            "Duración (min)", "Comentario general"]] = None

                df_workout_header.to_excel(writer, sheet_name=block_name, 
                                        startrow=start_row,
                                        index=False, header=False)
                start_row += df_workout_header.shape[0]
                df_workout.to_excel(writer, sheet_name=block_name,
                                    startrow=start_row,
                                    index=False)
                start_row += (df_workout.shape[0] + 2)

generate_program_excel(session, 4)

In [4]:
df_workout = pd.read_sql(
        session.query(Exercise.exercise_desc.label("Ejercicio"),
                      func.count(Workout_set.set_id).label("Series"),
                      Workout_set.no_reps.label("Repeticiones"),
                      Workout_set.weight.label("Peso (kg)"),
                      Workout_set.perc_rm.label("% 1RM"),
                      Workout_set.min_rpe.label("RPE mín."),
                      Workout_set.max_rpe.label("RPE máx."),
                      Workout_set.rest_min.label("Descanso (min)"))
                .group_by(Exercise.exercise_desc, 
                          Workout_set.weight)
                .filter(Exercise.exercise_id == Workout_set.exercise_id,
                        Workout_set.workout_id == 15)
                .order_by(Workout_set.workout_set_id)
                .statement,
        session.bind)

df_workout

# df_wod_header = pd.DataFrame([],
#                               index=["date", "desc", ""])
# df_wod_header

,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min)
0,press inclinado,4,10,60.0,60.0,None,7,2.0
1,apertura de mancuernas en banco inclinado,4,10,12.0,70.0,None,8,1.0
2,remo barra,4,10,60.0,60.0,None,8,2.0
3,remo con mancuernas en banco inclinado,4,12,20.0,80.0,None,8,1.0
4,press militar,4,10,40.0,80.0,None,8,2.0
5,press militar,4,15,30.0,70.0,None,9,2.0
6,face pull,4,12,30.0,80.0,None,8,1.0
7,face pull,4,15,20.0,70.0,None,8,1.0


In [9]:
session.query(Exercise.exercise_desc.label("Ejercicio"),
              Workout_set.set_id.label("Serie ID"),
              Workout_set.no_reps.label("Repeticiones"),
              Workout_set.weight.label("Peso (kg)"),
              Workout_set.perc_rm.label("% 1RM"),
              Workout_set.min_rpe.label("RPE mín."),
              Workout_set.max_rpe.label("RPE máx."),
              Workout_set.rest_min.label("Descanso (min)")) \
        .filter(Exercise.exercise_id == Workout_set.exercise_id,
                Workout_set.workout_id == 15) \
        .order_by(Workout_set.workout_set_id).all()

[('press inclinado', 1, 10, 60.0, 60.0, None, 7, 2.0),
 ('press inclinado', 2, 10, 60.0, 60.0, None, 7, 2.0),
 ('press inclinado', 3, 10, 60.0, 60.0, None, 7, 2.0),
 ('press inclinado', 4, 10, 60.0, 60.0, None, 7, 2.0),
 ('apertura de mancuernas en banco inclinado', 1, 10, 12.0, 70.0, None, 8, 1.0),
 ('apertura de mancuernas en banco inclinado', 2, 10, 12.0, 70.0, None, 8, 1.0),
 ('apertura de mancuernas en banco inclinado', 3, 10, 12.0, 70.0, None, 8, 1.0),
 ('apertura de mancuernas en banco inclinado', 4, 10, 12.0, 70.0, None, 8, 1.0),
 ('remo barra', 1, 10, 60.0, 60.0, None, 8, 2.0),
 ('remo barra', 2, 10, 60.0, 60.0, None, 8, 2.0),
 ('remo barra', 3, 10, 60.0, 60.0, None, 8, 2.0),
 ('remo barra', 4, 10, 60.0, 60.0, None, 8, 2.0),
 ('remo con mancuernas en banco inclinado', 1, 12, 20.0, 80.0, None, 8, 1.0),
 ('remo con mancuernas en banco inclinado', 2, 12, 20.0, 80.0, None, 8, 1.0),
 ('remo con mancuernas en banco inclinado', 3, 12, 20.0, 80.0, None, 8, 1.0),
 ('remo con mancuernas

In [43]:
PROGRAM_ID = 4
OUTPUT_DIR = "/mnt/c/Users/gonza/OneDrive/Gym/routines_log/"

try:
    program = session.query(Program).filter_by(program_id=PROGRAM_ID).one()
except NoResultFound:
    raise KeyError(f"Program_id ({PROGRAM_ID}) doesn't exist!")

# print(program.program_id, program.program_desc)
program_name = program.program_desc if program.program_desc else f"Program_{program.program_id}"
print(program_name)

with pd.ExcelWriter(OUTPUT_DIR + program_name + ".xlsx", engine="xlsxwriter") as writer:
    workbook = writer.book

    program_blocks = session.query(Block).filter_by(program_id=PROGRAM_ID).all()
    for program_block in program_blocks:
        block_name = program_block.block_desc
        print(block_name)
        # workbook.add_worksheet(block_name)
        start_row = 0
        for workout in program_block.workouts:
            df_workout_header = pd.DataFrame([workout.date_workout, workout.workout_desc], 
                                            index=["date", "desc"])
            display(df_workout_header)
            df_workout = pd.read_sql(
                session.query(Exercise.exercise_desc.label("Ejercicio"),
                            func.count(Workout_set.set_id).label("Series"),
                            Workout_set.no_reps.label("Repeticiones"),
                            Workout_set.weight.label("Peso (kg)"),
                            Workout_set.perc_rm.label("% 1RM"),
                            Workout_set.min_rpe.label("RPE mín."),
                            Workout_set.max_rpe.label("RPE máx."),
                            Workout_set.rest_min.label("Descanso (min)"))
                    .group_by(Exercise.exercise_desc)
                    .filter(Exercise.exercise_id == Workout_set.exercise_id,
                            Workout_set.workout_id == workout.workout_id)
                    .order_by(Workout_set.workout_set_id)
                    .statement,
                session.bind)
            
            # Add log fields (the No. Sets, No. Reps, Weight and RPE should be replaced
            # if needed)
            df_workout[["¿Hecho?", "Comentarios", "RPE total", 
                        "Duración (min)", "Comentario general"]] = None
            display(df_workout)

            df_workout_header.to_excel(writer, sheet_name=block_name, 
                                       startrow=start_row,
                                       index=False, header=False)
            start_row += df_workout_header.shape[0]
            df_workout.to_excel(writer, sheet_name=block_name,
                                startrow=start_row,
                                index=False)
            start_row += (df_workout.shape[0] + 2)

    # writer.save()             

Program_4
Micro 0 Gonzalo septiembre2021


,0
date,2021-09-06
desc,pierna


,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,Comentarios,RPE total,Duración (min),Comentario general
0,sentadilla,4,10,70.0,NaN,None,8,2.0,None,None,None,None,None
1,peso muerto,4,5,100.0,NaN,None,5,2.0,None,None,None,None,None
2,extensión de cuadriceps,4,10,50.0,NaN,None,8,1.0,None,None,None,None,None
3,femoral en maquina,4,10,20.0,NaN,None,7,1.0,None,None,None,None,None
4,abdominales crunch (solo 15º),5,15,2.0,70.0,None,7,1.0,None,None,None,None,None


,0
date,2021-09-07
desc,torso


,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,Comentarios,RPE total,Duración (min),Comentario general
0,press de banca,4,10,60.0,60.0,None,7,2.0,None,None,None,None,None
1,press plano mancuernas,4,10,25.0,70.0,None,7,2.0,None,None,None,None,None
2,dominadas libres,4,8,80.0,70.0,None,6,2.0,None,None,None,None,None
3,remo con barra,4,10,50.0,50.0,None,5,1.0,None,None,None,None,None
4,press francés barra olímpica,5,10,20.0,70.0,None,5,1.0,None,None,None,None,None


,0
date,2021-09-09
desc,acc pierna pst


,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,Comentarios,RPE total,Duración (min),Comentario general
0,peso muerto rumano,4,12,50.0,None,None,7,1.0,None,None,None,None,None
1,peso muerto rumano con mancuernas,4,10,20.0,None,None,8,1.0,None,None,None,None,None
2,hip thrust,4,8,100.0,None,None,6,2.0,None,None,None,None,None
3,sentadilla búlgara (2 mancuernas de 10),4,8,10.0,None,None,7,1.0,None,None,None,None,None
4,gemelo en multipower,5,12,60.0,None,None,6,1.0,None,None,None,None,None


,0
date,2021-09-11
desc,acc torso


,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,Comentarios,RPE total,Duración (min),Comentario general
0,press inclinado,4,12,50.0,50.0,None,6,2.0,None,None,None,None,None
1,press militar,4,12,30.0,70.0,None,8,2.0,None,None,None,None,None
2,press militar mancuernas sentado con respaldo,4,12,10.0,70.0,None,6,1.0,None,None,None,None,None
3,jalón al pecho,4,12,40.0,70.0,None,7,1.0,None,None,None,None,None
4,curl de biceps barra olímpica,4,12,20.0,70.0,None,8,1.0,None,None,None,None,None
5,curl de biceps mancuernas,4,10,10.0,70.0,None,7,1.0,None,None,None,None,None
6,face pull,5,15,10.0,70.0,None,8,1.0,None,None,None,None,None
7,rotación externa de hombro en polea,5,15,5.0,70.0,None,9,1.0,None,None,None,None,None


13

In [83]:
workbook.get_worksheet_by_name("Micro 0 Gonzalo septiembre2021")

In [3]:
session.query(Workout_set).all()

[<Workout_set(id=1,program=PHAT v2 (abr'21'),block=Volume_1,date=2021-04-09,week=1,day=3,exercise=press banca,set_id=1)>,
 <Workout_set(id=2,program=PHAT v2 (abr'21'),block=Volume_1,date=2021-04-09,week=1,day=3,exercise=press banca,set_id=2)>,
 <Workout_set(id=3,program=None,block=Micro 0 Gonzalo septiembre2021,date=2021-09-06,week=None,day=None,exercise=sentadilla,set_id=1)>,
 <Workout_set(id=4,program=None,block=Micro 0 Gonzalo septiembre2021,date=2021-09-06,week=None,day=None,exercise=sentadilla,set_id=2)>,
 <Workout_set(id=5,program=None,block=Micro 0 Gonzalo septiembre2021,date=2021-09-06,week=None,day=None,exercise=sentadilla,set_id=3)>,
 <Workout_set(id=6,program=None,block=Micro 0 Gonzalo septiembre2021,date=2021-09-06,week=None,day=None,exercise=sentadilla,set_id=4)>,
 <Workout_set(id=7,program=None,block=Micro 0 Gonzalo septiembre2021,date=2021-09-06,week=None,day=None,exercise=peso muerto,set_id=1)>,
 <Workout_set(id=8,program=None,block=Micro 0 Gonzalo septiembre2021,date=2

In [100]:
session.close()
engine.dispose()